In [ ]:
from __future__ import annotations

from pathlib import Path
import urllib.request

import geopandas as gpd
import matplotlib.pyplot as plt
from shapely.geometry import Point

In [ ]:
# Approx Elephant Butte Reservoir coordinates
EB_LON = -107.184
EB_LAT = 33.338

STATES_GEOJSON_URL = "https://raw.githubusercontent.com/PublicaMundi/MappingAPI/master/data/us-states.json"


In [ ]:
def download_if_missing(url: str, dest: Path) -> None:
    dest.parent.mkdir(parents=True, exist_ok=True)
    if dest.exists():
        return
    with urllib.request.urlopen(url) as resp:
        dest.write_bytes(resp.read())

In [ ]:
def make_locator_map(repo_root: Path) -> Path:
    states_path = repo_root / "data" / "external" / "us_states.geojson"
    download_if_missing(STATES_GEOJSON_URL, states_path)

    states = gpd.read_file(states_path)
    nm = states[states["name"] == "New Mexico"].copy()

    pt = gpd.GeoDataFrame(
        geometry=[Point(EB_LON, EB_LAT)],
        crs="EPSG:4326",
    )

    out_png = repo_root / "outputs" / "figures" / "locator_elephant_butte_nm.png"
    out_png.parent.mkdir(parents=True, exist_ok=True)

    fig, ax = plt.subplots(figsize=(6, 5))
    states.boundary.plot(ax=ax, linewidth=0.5)
    nm.boundary.plot(ax=ax, linewidth=2.0)

    pt.plot(ax=ax, markersize=60)  # marker

    # Optional label (small)
    ax.text(EB_LON + 0.25, EB_LAT, "Elephant Butte", fontsize=9)

    # zoom to NM with padding
    minx, miny, maxx, maxy = nm.total_bounds
    ax.set_xlim(minx - (maxx - minx) * 0.25, maxx + (maxx - minx) * 0.25)
    ax.set_ylim(miny - (maxy - miny) * 0.25, maxy + (maxy - miny) * 0.25)

    ax.set_title("Elephant Butte Reservoir — New Mexico")
    ax.set_axis_off()
    plt.tight_layout()
    fig.savefig(out_png, dpi=200)
    plt.close(fig)

    return out_png

In [ ]:
if __name__ == "__main__":
    repo_root = Path(r"C:\GIS\my_water_projects\openwater-shrinking-lake-monitor")
    out = make_locator_map(repo_root)
    print("✅ Saved:", out)